In [2]:
# Python SQL toolkit and Object Relational Mapper
import numpy as np
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func,inspect
from config import pgkey

In [5]:
stations = pd.read_csv("../Data/Nice_Ride_2017_Station_Locations.csv")
stations_df = pd.DataFrame(stations)
stations_df.head()

,ID,Number,Name,Latitude,Longitude,Total docks
0,1,30000,100 Main Street SE,44.984892,-93.256551,27
1,2,30001,25th Street & 33rd Ave S,44.957341,-93.223374,15
2,3,30002,Riverside Ave & 23rd Ave S,44.967115,-93.240149,15
3,4,30003,Plymouth Ave N & N Oliver Ave,44.991412,-93.306269,15
4,5,30004,11th Street & Hennepin,44.975340,-93.278690,23


In [29]:
bikes = pd.read_csv("../Data/Nice_ride_trip_history_2017_season.csv", low_memory=False)
bikes_df = pd.DataFrame(bikes)
bikes_df['Start Date'] = pd.to_datetime(bikes_df['Start Date'])
bikes_df.head()

,ID,Start Date,Start Time,Start station,Start station number,End Date,End Time,End station,End station number,Account type,Total duration (Seconds)
0,1,2017-11-05,21:45,Hennepin Ave & S Washington Ave,30184,11/5/17,22:02,Logan Park,30104,Member,1048
1,2,2017-11-05,22:45,Broadway Street N & 4th Street E,30122,11/5/17,22:26,Broadway Street N & 4th Street E,30122,Member,2513
2,3,2017-11-05,23:45,Dale Street & Grand Ave.,30106,11/5/17,22:13,N Milton Street & Summit Ave,30101,Member,1817
3,4,2017-11-05,0:45,Weisman Art Museum,30183,11/5/17,22:05,22nd Ave S & Franklin Ave,30014,Casual,1399
4,5,2017-11-05,1:45,South 2nd Street & 3rd Ave S,30030,11/5/17,21:44,6th Ave SE & University Ave,30088,Member,370


In [31]:
weather = pd.read_csv("../Data/WeatherDailyMinneapolis2017.csv", low_memory=False)
weather_df = pd.DataFrame(weather)
col = weather_df.loc[: , "TMAX":"TMIN"]
weather_df['TAVG'] = col.mean(axis=1)
weather_df.head()


,ID,STATION,NAME,DATE,PRCP,TMAX,TMIN,TAVG
0,1,USC00218450,"UNIVERSITY OF MN ST. PAUL, MN US",1/1/17,0.00,26,18,22.0
1,2,USC00218450,"UNIVERSITY OF MN ST. PAUL, MN US",1/2/17,0.00,31,20,25.5
2,3,USC00218450,"UNIVERSITY OF MN ST. PAUL, MN US",1/3/17,0.01,30,25,27.5
3,4,USC00218450,"UNIVERSITY OF MN ST. PAUL, MN US",1/4/17,0.00,25,-6,9.5
4,5,USC00218450,"UNIVERSITY OF MN ST. PAUL, MN US",1/5/17,0.01,2,-8,-3.0


In [13]:
num_stations = bikes_df['Start station'].nunique()
print(num_stations)

202


In [41]:
weather_clean = weather_df[['DATE', 'PRCP', 'TMAX', 'TMIN', 'TAVG']]
weather_clean = weather_clean.rename(columns={'DATE':'Start Date'})
weather_clean.head()
weather_clean['Start Date']=pd.to_datetime(weather_clean['Start Date'])
BigBikes = pd.merge(bikes_df, weather_clean, on="Start Date", how="left")
BigBikes.head()

,ID,Start Date,Start Time,Start station,Start station number,End Date,End Time,End station,End station number,Account type,Total duration (Seconds),PRCP,TMAX,TMIN,TAVG
0,1,2017-11-05,21:45,Hennepin Ave & S Washington Ave,30184,11/5/17,22:02,Logan Park,30104,Member,1048,0.1,41,33,37.0
1,2,2017-11-05,22:45,Broadway Street N & 4th Street E,30122,11/5/17,22:26,Broadway Street N & 4th Street E,30122,Member,2513,0.1,41,33,37.0
2,3,2017-11-05,23:45,Dale Street & Grand Ave.,30106,11/5/17,22:13,N Milton Street & Summit Ave,30101,Member,1817,0.1,41,33,37.0
3,4,2017-11-05,0:45,Weisman Art Museum,30183,11/5/17,22:05,22nd Ave S & Franklin Ave,30014,Casual,1399,0.1,41,33,37.0
4,5,2017-11-05,1:45,South 2nd Street & 3rd Ave S,30030,11/5/17,21:44,6th Ave SE & University Ave,30088,Member,370,0.1,41,33,37.0


In [55]:
BigBikes = BigBikes.reset_index()
#BigBikes=BigBikes.set_index('Start Date')
dates = BigBikes['Start Date'].unique()
for date in dates:
    account = [];
    account = BigBikes['Account type'].loc[(BigBikes['Start Date']==date)]
    print(account.count())

    

282
324
552
665
521
722
710
490
479
328
1084
1332
1027
1292
1544
1055
2270
1889
1846
1817
1704
1267
947
1935
1637
1696
1434
1547
2252
1595
1190
1781
2101
1572
1185
963
2803
2467
2088
2066
1764
1524
2007
2567
2210
2472
2313
2399
1180
2181
2500
2715
2591
2666
2605
2902
2273
2993
2733
2270
2218
1943
2118
3375
3022
2457
2625
2298
2328
2052
1881
695
1606
2441
2529
2292
1545
2426
3393
1921
1878
1436
2569
2105
1538
3836
2944
2000
1119
2830
3046
2418
3270
3005
1122
2923
2751
2777
3229
3903
3167
2752
2291
1954
2807
3184
4133
2835
2675
2468
2053
2485
3192
3361
3194
2677
2758
2706
2694
2774
4016
3150
2513
2588
4288
4147
4602
4193
3036
3286
1926
3285
2818
2910
2769
2655
1301
2846
2458
2635
2725
3028
2796
2647
2366
2318
1742
1513
2843
2721
2523
2471
2484
2403
2664
2926
2616
2303
2212
1439
1475
3330
3684
2630
2092
1756
1446
1947
665
211
1445
1194
843
1623
1726
2842
3558
2562
2561
1956
1902
1784
2610
3173
2611
2118
1803
1533
434
373
1664
1381
741
703
982
1603
2640
3459
2193
1000
782
920
1822
2137
148

In [52]:
members = BigBikes['Account type'].count()
members

AttributeError: 'DatetimeIndex' object has no attribute 'levels'